Title

Explanation


In [1]:
#Import necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt

#Read in table, explore format

sales = pd.read_csv('supermarket_sales - Sheet1.csv')
sales.head(3)


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4


In [2]:
#Clean up column names for consistency 

new_cols = {}

for col in sales.columns:
    clean = col.lower().replace(" ", "_").replace('%','').replace("5", "incr")
    new_cols[col] = clean
    
sales = sales.rename(columns = new_cols)
sales.head(3)


,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_incr,total,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4


In [3]:
# Explore for datatypes, missing values etc. 

sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   invoice_id               1000 non-null   object 
 1   branch                   1000 non-null   object 
 2   city                     1000 non-null   object 
 3   customer_type            1000 non-null   object 
 4   gender                   1000 non-null   object 
 5   product_line             1000 non-null   object 
 6   unit_price               1000 non-null   float64
 7   quantity                 1000 non-null   int64  
 8   tax_incr                 1000 non-null   float64
 9   total                    1000 non-null   float64
 10  date                     1000 non-null   object 
 11  time                     1000 non-null   object 
 12  payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross_margin_percentage  

Let's translate our date / time columns to the appropriate datatype. 

Looks like this dataset has been cleaned already - perfect! Note this won't always be the case, in such a scenario we would be required to clean and asses what to do with missing / imperfect values etc. 

In [4]:
#Datetime conversion 


In [5]:
sales.describe()
member_non_member = sales.groupby('customer_type').mean()
member_non_member.head()


,unit_price,quantity,tax_incr,total,cogs,gross_margin_percentage,gross_income,rating
customer_type,,,,,,,,
Member,56.206986,5.558882,15.609110,327.791305,312.182196,4.761905,15.609110,6.940319
Normal,55.135130,5.460922,15.148707,318.122856,302.974148,4.761905,15.148707,7.005210


In [6]:
# normalized_df = (sales - sales.min()) / (sales.max() - sales.min())
normalize_columns = ['unit_price', 'quantity', 'tax_incr', 'total','cogs', 
                     'gross_margin_percentage', 'gross_income','rating']

sales[normalize_columns] = (sales[normalize_columns] - sales[normalize_columns].min()
                           ) / (sales[normalize_columns].max() - sales[normalize_columns].min())
